# Normalizaçãodos dados

In [1]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import sqlite3 as sql
import seaborn as sns
import csv
import squarify 
from unidecode import unidecode
import os

from IPython.core.interactiveshell import InteractiveShell 
InteractiveShell.ast_node_interactivity="all"

# Tratamento dados TSE

In [2]:
!ls ../dados/tse/dados_limpos

detalhes_municipios.csv  detalhes_parlamentares.csv  output_voto_legenda.csv


### Tratamento dados parlamentares - cadastro base

In [3]:
#Dados de parlamentares
#enc="ISO-8859-3"
enc="Latin 1"
df_parlamentares=pd.read_csv("../dados/tse/dados_limpos/detalhes_parlamentares.csv", 
                             encoding=enc,
                             dtype={"NR_CPF_CANDIDATO":"object"},
                             header=0)

In [4]:
#df_parlamentares.sample(5)

In [5]:
#renomeando CPF
df_parlamentares.rename(columns={"NR_CPF_CANDIDATO":"CPF"}, inplace=True)
#df_parlamentares.info()

In [6]:
#df_parlamentares.columns

In [7]:
#Criando cadastro de parlamentares
df_parl_cadastro=df_parlamentares.copy()

In [8]:
# Drop nas colunas desnecessárias
df_parl_cadastro=df_parl_cadastro.drop(["DS_SIT_TOT_TURNO",'ST_REELEICAO',
                                        'ST_DECLARAR_BENS', 'CD_TIPO_BEM_CANDIDATO',
                                        'DS_TIPO_BEM_CANDIDATO', 'DS_BEM_CANDIDATO', 
                                        'VR_BEM_CANDIDATO'], axis=1)

In [9]:
# retirando linhas duplicadas do cadastro
df_parl_cadastro.drop_duplicates(inplace=True)
df_parl_cadastro.reset_index(drop=True, inplace=True)
#df_parl_cadastr.shape
#df_parl_cadastro.column

### Tratamento bens de parlamentares

In [10]:
#Criando tabela de bens de parmanetares
df_parl_bens=df_parlamentares.copy()
df_parl_bens=df_parl_bens.drop(["DS_SIT_TOT_TURNO",'ST_REELEICAO',"ANO_ELEICAO",
                                'SG_UE', 'DS_CARGO', 'SQ_CANDIDATO', 'NM_CANDIDATO',
                                'NM_URNA_CANDIDATO','NM_EMAIL', 'SG_PARTIDO',
                                'SG_UF_NASCIMENTO', 'NM_MUNICIPIO_NASCIMENTO', 
                                'DT_NASCIMENTO','NR_IDADE_DATA_POSSE', 'DS_GENERO',
                                'DS_GRAU_INSTRUCAO','DS_ESTADO_CIVIL', 'DS_COR_RACA', 
                                'DS_OCUPACAO', 'DS_SIT_TOT_TURNO','ST_REELEICAO'], 
                               axis=1)
df_parl_bens.reset_index(inplace=True, drop=True)
#df_parl_bens.shape
#df_parl_bens.info()

### Ajustando cadastro base com os bens dos parlamentares

In [11]:
# Criando valor total de bens dos parlamentares e colocando no cadastro
df_parl_cadastro["VL_BENS"]=0.
i=0
nparl=df_parl_cadastro.shape[0]
while i < nparl:
    cpf=df_parl_cadastro.iloc[i]["CPF"]
    total=df_parl_bens[df_parl_bens["CPF"]==cpf]["VR_BEM_CANDIDATO"].sum()
    #print("i-> {} CPF: {}, Valor: {}".format(i,cpf,total))
    #vl_bens[i]=total
    df_parl_cadastro.loc[i,"VL_BENS"]=total
    i+=1


In [12]:
#df_parl_cadastro.sample(5)

In [13]:
#Reordenando colunas
df_parl_cadastro=df_parl_cadastro[["CPF","NM_CANDIDATO","DT_NASCIMENTO","SG_UF_NASCIMENTO",
                  "NM_MUNICIPIO_NASCIMENTO", "DS_GENERO", "DS_GRAU_INSTRUCAO",
                  "DS_ESTADO_CIVIL", "DS_COR_RACA", "DS_OCUPACAO","VL_BENS",
                  "ANO_ELEICAO", "DS_CARGO", "SG_PARTIDO","SG_UE","SQ_CANDIDATO","NM_URNA_CANDIDATO",
                    "NR_IDADE_DATA_POSSE",'NM_EMAIL']]

In [14]:
df_parl_cadastro["NM_URNA_CANDIDATO"]=df_parl_cadastro["NM_URNA_CANDIDATO"].apply(lambda x:unidecode(x))

In [15]:
df_parl_cadastro.sample(5)

,CPF,NM_CANDIDATO,DT_NASCIMENTO,SG_UF_NASCIMENTO,NM_MUNICIPIO_NASCIMENTO,DS_GENERO,DS_GRAU_INSTRUCAO,DS_ESTADO_CIVIL,DS_COR_RACA,DS_OCUPACAO,VL_BENS,ANO_ELEICAO,DS_CARGO,SG_PARTIDO,SG_UE,SQ_CANDIDATO,NM_URNA_CANDIDATO,NR_IDADE_DATA_POSSE,NM_EMAIL
233,85702609620,ELIEL MARCIO DO CARMO,26/10/1973,MG,CONTAGEM,MASCULINO,SUPERIOR INCOMPLETO,CASADO(A),PRETA,VEREADOR,721246.26,2018,DEPUTADO FEDERAL,PSL,MG,130000616624,LEO MOTTA,45.0,VEREADORLEOMOTTA@GMAIL.COM
270,42844924034,PAULO ROBERTO SEVERO PIMENTA,19/03/1965,RS,SANTA MARIA,MASCULINO,SUPERIOR COMPLETO,CASADO(A),BRANCA,DEPUTADO,566034.12,2018,DEPUTADO FEDERAL,PT,RS,210000611383,PAULO PIMENTA,53.0,ANDREIATURNA@GMAIL.COM
543,23211253220,JOAO FERRARI JUNIOR,13/07/1967,PA,ORIXIMINA,MASCULINO,SUPERIOR COMPLETO,CASADO(A),PARDA,DEPUTADO,1796100.70,2018,DEPUTADO FEDERAL,PSD,PA,140000606382,JUNIOR FERRARI,51.0,NIVIA.BOTELHO@YAHOO.COM.BR
324,71943790582,ALESSANDRO VIEIRA,03/04/1975,RS,PASSO FUNDO,MASCULINO,SUPERIOR COMPLETO,CASADO(A),PARDA,POLICIAL CIVIL,900000.00,2018,SENADOR,REDE,SE,260000601810,DELEGADO ALESSANDRO VIEIRA,43.0,ALEVIEIRA1975@GMAIL.COM
27,00413473740,JOZIEL FERREIRA CARLOS,22/04/1966,RJ,SAO JOAO DE MERITI,MASCULINO,SUPERIOR COMPLETO,CASADO(A),PRETA,MILITAR REFORMADO,0.00,2018,DEPUTADO FEDERAL,PSL,RJ,190000614659,PROFESSOR JOZIEL,52.0,PROFESSORJOZIEL@GMAIL.COM


### Dados de votação nos municípios

In [16]:
#Dados de municipios e votacão
enc="ISO-8859-1"
#enc="Latin 1"
df_municipios=pd.read_csv("../dados/tse/dados_limpos/detalhes_municipios.csv", 
                             encoding=enc,
                            dtype={"NR_CPF_CANDIDATO":"object"},
                             header=0)

In [17]:
#renomeando CPF
df_municipios.rename(columns={"NR_CPF_CANDIDATO":"CPF"}, inplace=True)
#df_municipios.head(5)
#df_municipios.info()

In [18]:

#df_municipios.columns

In [19]:
df_parl_votacao=df_municipios[["CPF", "SG_UF", "CD_MUNICIPIO",
                            "VOTOS_TOTAL_MUN", 'CODIGO_IBGE']].copy()
df_parl_votacao.reset_index(inplace=True, drop=True)

In [20]:
df_parl_votacao.head()

,CPF,SG_UF,CD_MUNICIPIO,VOTOS_TOTAL_MUN,CODIGO_IBGE
0,93565968834,SP,68314,17542,3536505.0
1,00274003953,PR,79499,2969,4128807.0
2,01099561760,MS,98094,7110,5007695.0
3,26472058768,GO,96172,1582,5221007.0
4,00135968372,PI,12688,1252,2202653.0


In [24]:
# Percentual da votação total no municipio
df_parl_votacao["PERC_VOTOS"]=df_parl_votacao.groupby("CPF")["VOTOS_TOTAL_MUN"].apply(lambda x:(x / float(x.sum())*100).round(2))


In [25]:
df_parl_votacao[df_parl_votacao["CPF"]=="93565968834"][["VOTOS_TOTAL_MUN","PERC_VOTOS"]]
df_parl_votacao[df_parl_votacao["CPF"]=="93565968834"][["PERC_VOTOS"]].sum()


,VOTOS_TOTAL_MUN,PERC_VOTOS
0,17542,0.16
47,54457,0.49
54,41947,0.38
56,1789,0.02
57,1231,0.01
...,...,...
5540,1797,0.02
5546,1648,0.01
5559,1133,0.01
5567,908,0.01


PERC_VOTOS    100.05
dtype: float64

In [26]:
# Código com groupby para consolidar votação
#df_partido=df_partidos_votacao[df_partidos_votacao["SG_PARTIDO"]=="PSOL"]
#df_partido.groupby("SG_UF",as_index=False)["VOTOS_TOTAL_MUN"].sum().sort_values("VOTOS_TOTAL_MUN",ascending=False)

In [27]:
df_votacao=df_parl_cadastro[["CPF","DS_CARGO", "SG_PARTIDO","SG_UE"]].copy()
df_votacao.reset_index(inplace=True, drop=True)
df_votacao=pd.merge(df_votacao, df_parl_votacao, how="left", on="CPF")

In [28]:
#df_votacao.shape
#df_votacao.head()
df_votacao.head(5)

,CPF,DS_CARGO,SG_PARTIDO,SG_UE,SG_UF,CD_MUNICIPIO,VOTOS_TOTAL_MUN,CODIGO_IBGE,PERC_VOTOS
0,23019093104,SENADOR,PR,MT,MT,90042,1062,5100359.0,0.16
1,23019093104,SENADOR,PR,MT,MT,90166,1013,5108857.0,0.16
2,23019093104,SENADOR,PR,MT,MT,98337,2092,5103452.0,0.32
3,23019093104,SENADOR,PR,MT,MT,90670,113036,5103403.0,17.49
4,23019093104,SENADOR,PR,MT,MT,90905,1429,5101852.0,0.22


### Tabela consolidando votação dos partidos nos municípios (parlamentares)

In [29]:
df_partidos_votacao=df_votacao.groupby(["SG_PARTIDO","SG_UF","DS_CARGO","CODIGO_IBGE"], 
                                      as_index=False)["VOTOS_TOTAL_MUN"].sum()
df_partidos_votacao.shape

(94846, 5)

In [30]:
# Código com groupby para consolidar votação
#df_partido=df_partidos_votacao[df_partidos_votacao["SG_PARTIDO"]=="PSOL"]
#df_partido.groupby("SG_UF",as_index=False)["VOTOS_TOTAL_MUN"].sum().sort_values("VOTOS_TOTAL_MUN",ascending=False)

In [31]:
df_partidos_votacao.head()

,SG_PARTIDO,SG_UF,DS_CARGO,CODIGO_IBGE,VOTOS_TOTAL_MUN
0,AVANTE,AP,DEPUTADO FEDERAL,1600055.0,74
1,AVANTE,AP,DEPUTADO FEDERAL,1600105.0,89
2,AVANTE,AP,DEPUTADO FEDERAL,1600154.0,478
3,AVANTE,AP,DEPUTADO FEDERAL,1600204.0,22
4,AVANTE,AP,DEPUTADO FEDERAL,1600212.0,53


### Tabela consolidando votos em legenda nos municípios (legenda)

In [32]:
#Dados de municipios e votacão
enc="ISO-8859-1"
#enc="Latin 1"
df_legenda=pd.read_csv("../dados/tse/dados_limpos/output_voto_legenda.csv", 
                             encoding=enc,
                             sep=";",
                             header=0)

In [33]:
#df_legenda.head()
#df_legenda.shape

In [34]:
df_legendas_votacao=df_legenda.groupby(["SG_PARTIDO","SG_UF","DS_CARGO","CODIGO_IBGE"], 
                                      as_index=False)["VOTOS_TOTAL_MUN"].sum()
#df_legendas_votacao.shape
#df_legendas_votacao.head(5)


In [35]:
df_legendas_votacao["DS_CARGO"]=df_legendas_votacao["DS_CARGO"].apply(lambda x:x.upper())
df_legendas_votacao.sample(5)

,SG_PARTIDO,SG_UF,DS_CARGO,CODIGO_IBGE,VOTOS_TOTAL_MUN
199077,PT,GO,DEPUTADO FEDERAL,5214002.0,43
124147,PROS,MG,DEPUTADO FEDERAL,3113008.0,2
157797,PSD,PA,DEPUTADO FEDERAL,1503093.0,79
50760,PCB,MG,DEPUTADO FEDERAL,3149952.0,0
23930,MDB,MS,DEPUTADO FEDERAL,5001003.0,62


### Tabela cadastro com informações dos municípios

In [36]:
df_municipios=df_municipios[['CODIGO_IBGE','SG_UF','NM_MUNICIPIO',
                          'IDHM_2010', 'IDHM_RENDA_2010',
                          'IDHM_LONGEV_2010','IDHM_EDUC_2010']].copy()
df_municipios.drop_duplicates(inplace=True)
df_municipios.sort_values(["SG_UF","NM_MUNICIPIO"], inplace=True)
df_municipios.reset_index(inplace=True, drop=True)
df_municipios.head()

,CODIGO_IBGE,SG_UF,NM_MUNICIPIO,IDHM_2010,IDHM_RENDA_2010,IDHM_LONGEV_2010,IDHM_EDUC_2010
0,1200013.0,AC,ACRELANDIA,0.604,0.584,0.808,0.466
1,1200054.0,AC,ASSIS BRASIL,0.588,0.578,0.770,0.456
2,1200104.0,AC,BRASILEIA,0.614,0.619,0.770,0.485
3,1200138.0,AC,BUJARI,0.589,0.603,0.772,0.439
4,1200179.0,AC,CAPIXABA,0.575,0.601,0.794,0.398


## Salvando bases TSE em base sql

### Abrindo base sql

In [37]:
con = sql.connect("../dados/sql/base_completa.db")

### Salvando tabelas

In [38]:
df_parl_cadastro.to_sql("cadastro",con,if_exists="replace", index=False, index_label="CPF")
df_parl_bens.to_sql("parl_bens",con,if_exists="replace", index=False, index_label="CPF")
df_parl_votacao.to_sql("parl_votacao",con,if_exists="replace", index=False, index_label="CPF")
df_partidos_votacao.to_sql("partidos_votacao",con,if_exists="replace", index=False, index_label="SG_PARTIDO")
df_legendas_votacao.to_sql("legendas_votacao",con,if_exists="replace", index=False, index_label="SG_PARTIDO")
df_municipios.to_sql("municipios",con,if_exists="replace", index=False, index_label="CODIGO_IBGE")

### fechando base sql

In [39]:
con.close()

# Tratando diap

In [ ]:
df_diap=pd.read_csv("../dados/diap/politicos_diap_cpf.csv", dtype={"CPF":"object"})
#df_diap.info()

In [ ]:
df_diap.drop(columns=["nome","partido","cargo","estado","profissão"], inplace=True)
#df_diap.info()

In [ ]:
df_diap=df_diap[["CPF","mandatos","cabeças","debatedor","articulador","formulador","negociador"]].copy()
df_diap.head()

### Salvando tabela

In [ ]:
con = sql.connect("../dados/sql/base_completa.db")
df_diap.to_sql("diap",con,if_exists="replace", index=False, index_label="CPF")
con.close()

## Exemplo de abertura de base sql e carga em dataframe

In [ ]:
con = sql.connect("../dados/sql/base_completa.db")

In [ ]:
df=pd.read_sql("select * from cadastro",con)

In [ ]:
df.head()

In [ ]:
df_1=pd.read_sql("select * from bens where cpf=274003953",con)

In [ ]:
df_1

In [ ]:
con.close()

# Para verificar o conteúdo da base por uma ferramenta externa utilize
# https://sqlitebrowser.org/